In [1]:
import scipy
import numpy as np
import json

import spacy
import tensorflow
import keras
# from watson_developer_cloud import NaturalLanguageUnderstandingV1
# from watson_developer_cloud.natural_language_understanding_v1 \
#   import Features, KeywordsOptions, EntitiesOptions, SemanticRolesOptions

import nltk
import string

import inflection
import language_check # grammar checker
nlp = spacy.load('en_core_web_lg')

# set grammar checker
grammar_tool = language_check.LanguageTool('en-US')

#nltk.download('punkt')
#nltk.download('wordnet')

Using TensorFlow backend.


In [3]:
with open('article.txt', 'r') as file:
    doc = nlp(file.read())

entities = doc.ents
sentences = doc.sents
exceptList = ['DATE', 'MONEY', 'TIME', 'ORDINAL', 'QUANTITY', 'CARDINAL', 'PERCENT', 'ORDINAL']
for ent in entities:
    if ent.label_ not in exceptList:
        print(ent.text, ent.label_)

Apple ORG
Apple ORG
US GPE
Apple ORG
PetroChina ORG
Chinese NORP
Standard Oil ORG
Apple ORG
Apple ORG
Apple ORG
the Apple Watch ORG
Apple ORG
Nokia ORG
Apple ORG
Cisco GPE
Microsoft ORG
Cisco ORG
Apple ORG
Apple ORG
Apple ORG
Steve Jobs PERSON
Jobs PERSON
Apple Computer ORG
iPhone ORG
Apple ORG
iPhone ORG
Apple ORG
Macs ORG
iPads ORG
Apple ORG
Google Maps LOC
YouTube GPE
Apple ORG
iCloud ORG
Apple ORG
iPhone ORG
iPad ORG
Mac ORG
Apple ORG
Samsung ORG
LG ORG
Huawei ORG
Xiaomi ORG
Amazon ORG
Google ORG
Amazon Web Services ORG
Google ORG
Bill Gates PERSON
Microsoft ORG
Apple ORG
Apple ORG
IBM ORG
Exxon ORG
Apple ORG
iPhone ORG
the United States GPE
Apple ORG
Apple ORG
America GPE


In [ ]:
# #Load credentials from file (Store credentials in json format)
# with open('credentials.json') as f:
#     data = json.load(f)
# url = data["url"]
# username = data["username"]
# password = data["password"]

In [ ]:
# natural_language_understanding = NaturalLanguageUnderstandingV1(
#   username=username,
#   password=password,
#   version='2018-03-16')

# response = natural_language_understanding.analyze(
#     url='https://www.nytimes.com/2018/07/16/opinion/trump-and-putin-vs-america.html?action=click&pgtype=Homepage&clickSource=story-heading&module=opinion-c-col-left-region&region=opinion-c-col-left-region&WT.nav=opinion-c-col-left-region',
#     language='en',
#     features=Features(
#         keywords=KeywordsOptions(
#             sentiment=False,
#             emotion=False,
#             limit=20),
#     entities=EntitiesOptions(
#         sentiment=False,
#             emotion=False,
#             limit=50),
#     semantic_roles=SemanticRolesOptions()
#   ))
# entities = response['entities']
# keywords = response['keywords']
# semantic = response['semantic_roles']

In [4]:
# Question 1
# Extract keywords and entities
# define type of words
# create questions
def Q1(x):
    return {
        'PERSON': "Who is ",
        'GPE': "Where is "
    }.get(x, "What is ")

keyEntities = set()
duplicates = set()

with open("Questions1.txt", "w") as file:
    for ent in entities:
        if ent.label_ not in exceptList and ent.text not in duplicates:
            # make question
            text = Q1(ent.label_) + ent.text + "?"
            # check and fix grammar of question
            matches = grammar_tool.check(text)
            correct_text = language_check.correct(text, matches)
            # write to file and add to unique set
            file.write("%s\n" % correct_text)
            keyEntities.add(ent)
            duplicates.add(ent.text)

In [5]:
for a in keyEntities:
    print(a.text)

Jobs
America
Steve Jobs
YouTube
Bill Gates
Huawei
Google
Cisco
Microsoft
Samsung
Google Maps
Xiaomi
US
Chinese
Exxon
LG
Standard Oil
iCloud
Nokia
PetroChina
Amazon Web Services
Macs
iPads
iPhone
Amazon
IBM
the United States
the Apple Watch
iPad
Apple Computer
Apple
Mac


In [ ]:
#Question 2
# Extract keywords and entities
# define plurality of keywords using nltk
# create question What are? Who are? 
# TODO. How to determine plurality of word?
# TODO. If condition does not suffice, what else?

In [ ]:
#Question 3

duplicates = set()

with open("refined_named_entities.txt", 'r') as file: # create tokens from known named entities
    doc = nlp(file.read()) 
# for t in doc.ents:
#     print(t.text)
    
with open("Questions3.txt", "w") as file: # create questions based on similarity between 'entities' and 'known named entities'
    for key in keyEntities:
        for token in doc.ents:
            tokenSimilarity = token.similarity(key)
            if tokenSimilarity > 0.6 and tokenSimilarity != 1 and token.text not in duplicates and len(token.text) > 2:
                file.write("What is the difference between %s and %s?\n" % (token.text, key.text))
                duplicates.add(token.text)

In [ ]:
for sentence in sents:
    

In [7]:
# Question 4
# Why (factual statement)?
# TODO. How to determine which sentences are important enough to negate? TEXTSUM, sumy NOTWORKING

# Question 5
# What if (negated statements)?
# TODO. How to negate sentences? add not, find antonym
# TODO. How to determine which sentences are important enough to negate? summarize? gensim, pyteaser, pytextrank, TEXTSUM, sumy

Q4 = []
Q5 = []

listOfPlurals = ["they", "some", "most", "we"]
translator = str.maketrans('', '', string.punctuation) #strip punctuations

for sentence in semantic:
    if len(sentence) == 4:
        verb = sentence['action']['normalized']
        subj = sentence['subject']['text'].capitalize()
        obj = sentence['object']['text']
        if verb is not "s" and verb != "be":
            plurality = subj is not inflection.singularize(subj) or subj in listOfPlurals
            if plurality:
                Q4.append("Why do " + subj + " " + verb + " " + obj + "?")
            else:
                Q4.append("Why does " + subj + " " + verb + " " + obj + "?")
            Q5.append("What if " + subj + " did not " + verb + " " + obj + "?")
        elif verb == "be":
            plurality = subj is not inflection.singularize(subj) or subj in listOfPlurals
            if plurality:
                Q4.append("Why are " + subj + " " + obj + "?")
                Q5.append("What if " + subj + " were not " + obj + "?")
            else:
                Q4.append("Why is " + subj + " " + obj + "?")
                Q5.append("What if " + subj + " was not " + obj + "?")
        
with open("Questions4.txt", "w", encoding='UTF-8') as file:
    for q in Q4:
        grammarCheckAndWrite(q, file)

with open("Questions5.txt", "w", encoding='UTF-8') as file:
    for q in Q5:
        grammarCheckAndWrite(q, file)

# TODO: Check relevance to text
# TODO: lets do grammar check first and delete or fix the grammar before comparing relevance to text


In [ ]:
def grammarCheckAndWrite(question, file):
    matches = grammar_tool.check(question)
    corrected_q = language_check.correct(question, matches)
    file.write("%s\n" % corrected_q)